In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(root=data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dir = '../Image Data/Paintings/training'
val_dir = '../Image Data/Paintings/testing'
train_dataset = CustomDataset(train_dir, transform=transform)
val_dataset = CustomDataset(val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
num_classes = 8
model = CustomResNet(num_classes)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
print_every = 7 

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    minibatch_counter = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        minibatch_counter += 1
        if minibatch_counter % print_every == 0:
            minibatch_loss = running_loss / total
            minibatch_accuracy = correct / total
            print(f'>>>>> Minibatch [{minibatch_counter}/{len(train_loader)}], Loss: {minibatch_loss:.4f}, Accuracy: {minibatch_accuracy:.4f}')

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct / total
    print(f'Train [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    minibatch_counter = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            minibatch_counter += 1
            if minibatch_counter % print_every == 0:
                minibatch_loss = val_loss / val_total
                minibatch_accuracy = val_correct / val_total
                print(f'>>>>> Minibatch [{minibatch_counter}/{len(val_loader)}], Loss: {minibatch_loss:.4f}, Accuracy: {minibatch_accuracy:.4f}')

    val_epoch_loss = val_loss / len(val_dataset)
    val_epoch_accuracy = val_correct / val_total
    print(f'Validation [{epoch+1}/{num_epochs}], Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.4f}')

print('Fine-tuning finished.')
torch.save(model.state_dict(), 'Painting.pth')

>>>>> Minibatch [7/28], Loss: 1.9670, Accuracy: 0.2746
>>>>> Minibatch [14/28], Loss: 1.8444, Accuracy: 0.3460
>>>>> Minibatch [21/28], Loss: 1.7448, Accuracy: 0.4048
>>>>> Minibatch [28/28], Loss: 1.6486, Accuracy: 0.4543
Train [1/10], Loss: 1.6486, Accuracy: 0.4543
>>>>> Minibatch [7/12], Loss: 1.2614, Accuracy: 0.6116
Validation [1/10], Loss: 1.2720, Accuracy: 0.6263
>>>>> Minibatch [7/28], Loss: 1.1736, Accuracy: 0.7031
>>>>> Minibatch [14/28], Loss: 1.1371, Accuracy: 0.6987
>>>>> Minibatch [21/28], Loss: 1.0863, Accuracy: 0.7098
>>>>> Minibatch [28/28], Loss: 1.0587, Accuracy: 0.7158
Train [2/10], Loss: 1.0587, Accuracy: 0.7158
>>>>> Minibatch [7/12], Loss: 0.9714, Accuracy: 0.7098
Validation [2/10], Loss: 0.9736, Accuracy: 0.7164
>>>>> Minibatch [7/28], Loss: 0.8498, Accuracy: 0.7991
>>>>> Minibatch [14/28], Loss: 0.8452, Accuracy: 0.7857
>>>>> Minibatch [21/28], Loss: 0.8439, Accuracy: 0.7842
>>>>> Minibatch [28/28], Loss: 0.8221, Accuracy: 0.7900
Train [3/10], Loss: 0.8221, Acc

In [9]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

num_classes = 8  
model = CustomResNet(num_classes)
model.load_state_dict(torch.load('Painting.pth'))
model.eval()

train_dataset = ImageFolder(root='../Image Data/Paintings/training', transform=transform)
class_names = train_dataset.classes
def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    predicted_class = class_names[predicted.item()] 
    return predicted_class

image_paths = ['../Image Data/Paintings/training/Kangra/p1.jpg', '../Image Data/Paintings/training/Madhubani/p1.jpg']  
for image_path in image_paths:
    predicted_class = predict_image(image_path)
    print(f'Image: {image_path}, Predicted Class: {predicted_class}')


Image: ../Image Data/Paintings/training/Kangra/p1.jpg, Predicted Class: Kangra
Image: ../Image Data/Paintings/training/Madhubani/p1.jpg, Predicted Class: Madhubani
